In [1]:
from theano.sandbox import cuda

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


In [2]:
import utils; reload(utils)
from utils import *

Using Theano backend.


In [3]:
ratings = pd.read_csv('data/ml-latest-small/ratings.csv')

In [4]:
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [5]:
movie_names = pd.read_csv('data/ml-latest-small/movies.csv')

In [6]:
movie_names

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [7]:
# changing the movie and user ids so they are continuous integers
users = ratings.userId.unique()
movies = ratings.movieId.unique()

In [8]:
userid2idx = {o:i for i,o in enumerate(users)}

In [9]:
userid2idx

{1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 7: 6,
 8: 7,
 9: 8,
 10: 9,
 11: 10,
 12: 11,
 13: 12,
 14: 13,
 15: 14,
 16: 15,
 17: 16,
 18: 17,
 19: 18,
 20: 19,
 21: 20,
 22: 21,
 23: 22,
 24: 23,
 25: 24,
 26: 25,
 27: 26,
 28: 27,
 29: 28,
 30: 29,
 31: 30,
 32: 31,
 33: 32,
 34: 33,
 35: 34,
 36: 35,
 37: 36,
 38: 37,
 39: 38,
 40: 39,
 41: 40,
 42: 41,
 43: 42,
 44: 43,
 45: 44,
 46: 45,
 47: 46,
 48: 47,
 49: 48,
 50: 49,
 51: 50,
 52: 51,
 53: 52,
 54: 53,
 55: 54,
 56: 55,
 57: 56,
 58: 57,
 59: 58,
 60: 59,
 61: 60,
 62: 61,
 63: 62,
 64: 63,
 65: 64,
 66: 65,
 67: 66,
 68: 67,
 69: 68,
 70: 69,
 71: 70,
 72: 71,
 73: 72,
 74: 73,
 75: 74,
 76: 75,
 77: 76,
 78: 77,
 79: 78,
 80: 79,
 81: 80,
 82: 81,
 83: 82,
 84: 83,
 85: 84,
 86: 85,
 87: 86,
 88: 87,
 89: 88,
 90: 89,
 91: 90,
 92: 91,
 93: 92,
 94: 93,
 95: 94,
 96: 95,
 97: 96,
 98: 97,
 99: 98,
 100: 99,
 101: 100,
 102: 101,
 103: 102,
 104: 103,
 105: 104,
 106: 105,
 107: 106,
 108: 107,
 109: 108,
 110: 109,
 111: 11

In [10]:
movieid2idx = {o:i for i,o in enumerate(movies)}

In [11]:
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

In [12]:
ratings

,userId,movieId,rating,timestamp
0,0,0,2.5,1260759144
1,0,1,3.0,1260759179
2,0,2,3.0,1260759182
3,0,3,2.0,1260759185
4,0,4,4.0,1260759205
5,0,5,2.0,1260759151
6,0,6,2.0,1260759187
7,0,7,2.0,1260759148
8,0,8,3.5,1260759125
9,0,9,2.0,1260759131


In [13]:
ratings.shape

(100004, 4)

In [14]:
# split into training and test
msk = np.random.choice([True, False], size = 100004, p=[0.8,0.2])

In [15]:
sum(msk)

80116

In [16]:
training_data = ratings[msk]
val_data = ratings[~msk]

In [17]:
val_data.shape

(19888, 4)

In [18]:
??Embedding

https://keras.io/layers/embeddings/

# Dot product

https://keras.io/getting-started/functional-api-guide/

In [19]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')

In [20]:
user_in

user_in

In [21]:
type(user_in)

theano.tensor.var.TensorVariable

In [22]:
Embedding(ratings.userId.nunique(), 50, input_length=1, W_regularizer=l2(1e-4))

In [23]:
user_embedding = Embedding(training_data.userId.nunique(), 50, input_length=1, W_regularizer=l2(1e-4))(user_in)

In [24]:
user_embedding

Reshape{3}.0

In [25]:
type(user_embedding)

theano.tensor.var.TensorVariable

In [26]:
type(Sequential())

keras.models.Sequential

In [27]:
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')

In [28]:
movie_embedding = Embedding(ratings.movieId.nunique(), 50, input_length=1, W_regularizer=l2(1e-4))(movie_in)

In [29]:
x = merge([user_embedding, movie_embedding], mode = 'dot')

In [30]:
x

Reshape{3}.0

In [31]:
x = Flatten()(x)

In [32]:
x

Reshape{2}.0

In [33]:
model = Model([user_in, movie_in], x)

https://keras.io/models/model/

In [34]:
model.compile(Adam(0.001), loss='mse')

In [35]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_in (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
movie_in (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 1, 50)         33550       user_in[0][0]                    
____________________________________________________________________________________________________
embedding_3 (Embedding)          (None, 1, 50)         453300      movie_in[0][0]                   
___________________________________________________________________________________________

In [36]:
model.fit([training_data.userId, training_data.movieId], training_data.rating,
          batch_size=64, nb_epoch=1, verbose=2,
          validation_data=([val_data.userId, val_data.movieId], val_data.rating))

Train on 80116 samples, validate on 19888 samples
Epoch 1/1
7s - loss: 9.9186 - val_loss: 4.2839


In [37]:
model.optimizer.lr=0.01

In [39]:
model.fit([training_data.userId, training_data.movieId], training_data.rating,
          batch_size=64, nb_epoch=1, verbose=2,
          validation_data=([val_data.userId, val_data.movieId], val_data.rating))

Train on 80116 samples, validate on 19888 samples
Epoch 1/1
7s - loss: 3.1207 - val_loss: 2.8445


In [40]:
model.fit([training_data.userId, training_data.movieId], training_data.rating,
          batch_size=64, nb_epoch=2, verbose=2,
          validation_data=([val_data.userId, val_data.movieId], val_data.rating))

Train on 80116 samples, validate on 19888 samples
Epoch 1/2
7s - loss: 2.4113 - val_loss: 2.6472
Epoch 2/2
7s - loss: 2.2535 - val_loss: 2.6020


In [41]:
model.optimizer.lr=0.001
model.fit([training_data.userId, training_data.movieId], training_data.rating,
          batch_size=64, nb_epoch=6, verbose=2,
          validation_data=([val_data.userId, val_data.movieId], val_data.rating))

Train on 80116 samples, validate on 19888 samples
Epoch 1/6
7s - loss: 2.1946 - val_loss: 2.5850
Epoch 2/6
7s - loss: 2.1625 - val_loss: 2.5844
Epoch 3/6
7s - loss: 2.1375 - val_loss: 2.5812
Epoch 4/6
7s - loss: 2.1128 - val_loss: 2.5873
Epoch 5/6
7s - loss: 2.0879 - val_loss: 2.5890
Epoch 6/6
7s - loss: 2.0627 - val_loss: 2.5946


# Bias

In [47]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg))(inp)

In [48]:
user_in, u = embedding_input('user_in', ratings.userId.nunique(), 50, 1e-4)
movie_in, m = embedding_input('movie_in', ratings.movieId.nunique(), 50, 1e-4)

In [49]:
def create_bias(inp, n_in):
    x = Embedding(n_in, 1, input_length=1)(inp)
    return Flatten()(x)

In [58]:
u_b = create_bias(user_in, ratings.userId.nunique())
m_b = create_bias(movie_in, ratings.movieId.nunique())

In [59]:
x = merge([u, m], mode = 'dot')

In [60]:
x = Flatten()(x)

In [61]:
x = merge([x, u_b], mode = 'sum')

In [62]:
x = merge([x, m_b], mode = 'sum')

In [65]:
model = Model([user_in, movie_in], x)

In [66]:
model.compile(Adam(0.001), loss='mse')

In [67]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_in (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
movie_in (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_6 (Embedding)          (None, 1, 50)         33550       user_in[0][0]                    
____________________________________________________________________________________________________
embedding_7 (Embedding)          (None, 1, 50)         453300      movie_in[0][0]                   
___________________________________________________________________________________________

In [69]:
model.fit([training_data.userId, training_data.movieId], training_data.rating,
          batch_size=64, verbose=2, nb_epoch=1,
         validation_data=([val_data.userId, val_data.movieId], val_data.rating))

Train on 80116 samples, validate on 19888 samples
Epoch 1/1
6s - loss: 8.9597 - val_loss: 3.5969


In [71]:
model.optimizer.lr=0.01
model.fit([training_data.userId, training_data.movieId], training_data.rating,
          batch_size=64, verbose=2, nb_epoch=6,
         validation_data=([val_data.userId, val_data.movieId], val_data.rating))

Train on 80116 samples, validate on 19888 samples
Epoch 1/6
6s - loss: 2.5970 - val_loss: 2.3228
Epoch 2/6
6s - loss: 1.9807 - val_loss: 2.1142
Epoch 3/6
6s - loss: 1.8173 - val_loss: 2.0222
Epoch 4/6
6s - loss: 1.7249 - val_loss: 1.9563
Epoch 5/6
6s - loss: 1.6464 - val_loss: 1.8829
Epoch 6/6
6s - loss: 1.5732 - val_loss: 1.8187


In [72]:
model.optimizer.lr=0.001
model.fit([training_data.userId, training_data.movieId], training_data.rating,
          batch_size=64, verbose=2, nb_epoch=6,
         validation_data=([val_data.userId, val_data.movieId], val_data.rating))

Train on 80116 samples, validate on 19888 samples
Epoch 1/6
6s - loss: 1.5030 - val_loss: 1.7575
Epoch 2/6
6s - loss: 1.4348 - val_loss: 1.6957
Epoch 3/6
6s - loss: 1.3677 - val_loss: 1.6354
Epoch 4/6
6s - loss: 1.3026 - val_loss: 1.5786
Epoch 5/6
6s - loss: 1.2399 - val_loss: 1.5263
Epoch 6/6
6s - loss: 1.1800 - val_loss: 1.4764


In [73]:
model.fit([training_data.userId, training_data.movieId], training_data.rating,
          batch_size=64, verbose=2, nb_epoch=5,
         validation_data=([val_data.userId, val_data.movieId], val_data.rating))

Train on 80116 samples, validate on 19888 samples
Epoch 1/5
6s - loss: 1.1224 - val_loss: 1.4281
Epoch 2/5
6s - loss: 1.0685 - val_loss: 1.3873
Epoch 3/5
6s - loss: 1.0163 - val_loss: 1.3435
Epoch 4/5
6s - loss: 0.9677 - val_loss: 1.3056
Epoch 5/5
6s - loss: 0.9213 - val_loss: 1.2700


# Neural net

In [74]:
user_in, u = embedding_input('user_in', ratings.userId.nunique(), 50, 1e-4)
movie_in, m = embedding_input('movie_in', ratings.movieId.nunique(), 50, 1e-4)

In [75]:
x = merge([u, m], mode = 'concat')
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(70, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)
nn = Model([user_in, movie_in], x)
nn.compile(Adam(0.001), loss='mse')

In [76]:
nn.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_in (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
movie_in (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_12 (Embedding)         (None, 1, 50)         33550       user_in[0][0]                    
____________________________________________________________________________________________________
embedding_13 (Embedding)         (None, 1, 50)         453300      movie_in[0][0]                   
___________________________________________________________________________________________

In [77]:
nn.fit([training_data.userId, training_data.movieId], training_data.rating,
       batch_size=64, verbose=2, nb_epoch=5,
       validation_data=([val_data.userId, val_data.movieId], val_data.rating))

Train on 80116 samples, validate on 19888 samples
Epoch 1/5
8s - loss: 2.4699 - val_loss: 0.9121
Epoch 2/5
8s - loss: 1.4852 - val_loss: 0.8783
Epoch 3/5
8s - loss: 1.2341 - val_loss: 0.8515
Epoch 4/5
8s - loss: 1.0430 - val_loss: 0.8546
Epoch 5/5
8s - loss: 0.9138 - val_loss: 0.8333
